# 1A. Minkowski Functionals

This notebook demonstrates some methods of computing the four Minkowski functionals (in 3D). The four Minkowski functionals are:

1. Volume
2. Surface Area
3. Mean Curvature
4. Total Integral Curvature/Euler Characteristic

## Import some packages

In [1]:
# Image processsing
import skimage
from skimage.morphology import ball

# Image visualization
import pyvista as pv
pv.set_jupyter_backend('server')
import matplotlib.pyplot as plt
import tifffile

# Image quantification
import numpy as np
from quantimpy import minkowski as mk
import porespy as ps
import trimesh

# Other utility functions
import os
# TODO: be smarter about changing directory
os.chdir('../')
print(os.getcwd())
import glob
import src.utils
# if glob.glob('data/*.tif') == []:
#     src.utils.get_datafiles()
import ipywidgets as widgets, interactive
from IPython.display import display, clear_output



/Users/bchang/Documents/porescale_quantification_workshop


## Loading in our data

We load in TIFF images from the data directory. This directory contains segmented x-ray microtomography data from the Network Generation Comparison Forum and is available on the Digital Rocks Portal (https://www.digitalrocksportal.org/projects/16). 

For the purposes of this workshop, we have preprocessed the data to be in TIFF file format with solid and pore labeled as 0 and 1, respectively.

In [2]:
img_list = glob.glob('data/*.tif')
print(img_list)
segmented_Gambier, segmented_bead_pack_512, segmented_lrc32,   segmented_castle_512 = [tifffile.imread(tif_img) for tif_img in img_list]

fig, ax = plt.subplots(nrows=2, ncols=2,figsize=(8,8))
ax = ax.flatten()

ax[0].imshow(segmented_lrc32[0,:,:], cmap='gray')
ax[0].set_title('LRC32 Sand Pack',fontsize=14)

ax[1].imshow(segmented_Gambier[0,:,:], cmap='gray')
ax[1].set_title('Gambier Limestone',fontsize=14)

ax[2].imshow(segmented_bead_pack_512[0,:,:], cmap='gray')
ax[2].set_title('Glass Bead Pack',fontsize=14)

im = ax[3].imshow(segmented_castle_512[0,:,:], cmap='gray')
ax[3].set_title('Castlegate Sandstone',fontsize=14)

fig.colorbar(im, ax=ax.tolist())

plt.show()

['data/mtgambier.tif', 'data/beadpack.tif', 'data/sandpack.tif', 'data/castlegate.tif']


## 3D Visualization

Here, we view our segmented image in 3D using the PyVista 3D visualization library.

<font color='red'>**Warning:**</font> These images are $512^3$ voxels and can be quite large to try to visualize on a local machine. By default, we plot a $100^3$ subset of the full images, but you can try to plot the entire image if your machine allows.

In [6]:
widget = widgets.Dropdown()
widget.options=['Beadpack', 'Sandpack', 'Mt. Gambier Limestone', 'Castlegate Sandstone']
widget.description='Sample: '
widget.value='Beadpack'

subset = widgets.Checkbox(value=True,
                           description='Plot Subset',)


display(widget, subset)



def plot_sample(*args):
    
    sample_dict = {'Sandpack': segmented_lrc32,
                  'Mt. Gambier Limestone': segmented_Gambier, 
                   'Beadpack': segmented_bead_pack_512, 
                   'Castlegate Sandstone': segmented_castle_512}
    

    
    plotter_obj = pv.Plotter(lighting='three lights')#notebook=True, off_screen=False, lighting='three lights')

    # Set background colors
    plotter_obj.set_background(color='w')
#     plotter_obj.add_light(pv.Light(light_type='headlight'))

    # Set font colors and sizes
    pv.global_theme.font.color = 'black'
    pv.global_theme.font.size = 18
    pv.global_theme.font.label_size = 14
    
    clear_output(wait=True)
    pv.set_jupyter_backend('server')
    display(widget, subset)
    
    sample = sample_dict[widget.value]
    if subset.value:
        sample = sample[206:306, 206:306, 206:306]
    
    pv_image_obj = pv.wrap(sample)
    contours = pv_image_obj.contour(isosurfaces=[0.5])
    plotter_obj.add_mesh(contours, opacity=1, color=(200 / 255, 181 / 255, 152 / 255), diffuse=0.75, ambient=0.15)
    
    
    plotter_obj.show(jupyter_backend='server')
    
    print(widget.value)
    
plot_sample()
widget.observe(plot_sample, 'value')
subset.observe(plot_sample, 'value')

    

Dropdown(description='Sample: ', options=('Beadpack', 'Sandpack', 'Mt. Gambier Limestone', 'Castlegate Sandsto…

Checkbox(value=True, description='Plot Subset')

Widget(value='<iframe src="http://localhost:50415/index.html?ui=P_0x33c8ff510_6&reconnect=auto" class="pyvista…

Beadpack


## Compute the Minkowski Functionals

For the following introductory demonstrations, we will use a single solid sphere sample as the motivating example. At the end, we will have the opportunity to try the functions on the other samples.

Generally speaking, the Minkowski functionals can be computed using voxel methods or mesh methods. We will explore both here using the Quantimpy (voxelized) and the Trimesh/Skimage (mesh) libraries.

#### In the following cell, we create and visualize a solid sphere


In [4]:
radius = 48
side_length = 128

sphere = np.zeros([side_length, side_length, side_length], dtype=np.uint8)
id_a = side_length // 2 - radius
id_b = side_length // 2 + radius + 1
sphere[id_a:id_b, id_a:id_b, id_a:id_b] = ball(radius, dtype=np.uint8)

plotter_obj = pv.Plotter(notebook=True, off_screen=False, lighting='three lights')

# Set background colors
plotter_obj.set_background(color='w')
#     plotter_obj.add_light(pv.Light(light_type='headlight'))

# Set font colors and sizes
pv.global_theme.font.color = 'black'
pv.global_theme.font.size = 18
pv.global_theme.font.label_size = 14

pv.set_jupyter_backend('server')

pv_image_obj = pv.wrap(sphere)
contours = pv_image_obj.contour(isosurfaces=[0.5])
plotter_obj.add_mesh(contours, opacity=1, color=(200 / 255, 181 / 255, 152 / 255), diffuse=0.75, ambient=0.15)


plotter_obj.show(jupyter_backend='server')

Widget(value='<iframe src="http://localhost:50415/index.html?ui=P_0x2c3e3e4d0_3&reconnect=auto" class="pyvista…

### Before we begin the calculation, we create a mesh using the marching cubes algorithm implemented in the PoreSpy library

In [6]:
example_mesh = ps.tools.mesh_region(sphere)

#### We also create a function to compare our package computations with the analytical solution


In [5]:
def analytical_solution(radius):
    volume_analytical = 4/3 * np.pi * radius**3
    surface_area_analytical = 4 * np.pi * radius**2
    mean_curvature_analytical = 4 * np.pi * radius
    euler_analytical = 1.0
    
    return volume_analytical, surface_area_analytical, mean_curvature_analytical, euler_analytical

def compute_error(true, computed_measures):
    return [np.abs((true - compute) / true)*100 for compute in computed_measures]

volume_analytical, surface_area_analytical, mean_curvature_analytical, euler_analytical = analytical_solution(radius)

## 1. Volume Calculation

Considering a 3D body, $X$, with a smooth boundary, $\delta X$, the first Minkowski functional (volume) can be computed as:
$$ M_0(X) = \int_X dv $$

In practice, volume is the most straightforward of the Minkowski functionals set to compute.

Here, we compare a few different methods and packages for computing the volume of the pore space, split into voxelized methods and mesh methods:

1. Voxel Counting
2. Quantimpy (Voxelized)
3. Trimesh (Mesh)



In [6]:
def compute_volume(image):
    # Simple voxel counting
    voxel_counting = np.sum(image == 1)
    
    # Mesh Volume
    mesh_measures = np.abs(ps.metrics.mesh_volume(image))
    
    # Quantimpy
    quantimpy_measures = mk.functionals(image.astype(bool))[0]
    
    return voxel_counting, quantimpy_measures, mesh_measures

volume_measures = compute_volume(sphere)
volume_errors = compute_error(volume_analytical, volume_measures)

print("Volume Calculations:")
print(f"\tAnalytical Solution: {volume_analytical:0.3f}")
print(f"\tVoxel Counting: {volume_measures[0]:0.3f} \t Rel. Error: {volume_errors[0]:0.3f}%")
print(f"\tQuantimpy: {volume_measures[1]:0.3f} \t\t Rel. Error: {volume_errors[1]:0.3f}%")
print(f"\tMesh: {volume_measures[2]:0.3f} \t\t Rel. Error: {volume_errors[2]:0.3f}%")


Volume Calculations:
	Analytical Solution: 463246.686
	Voxel Counting: 462781.000 	 Rel. Error: 0.101%
	Quantimpy: 462781.000 		 Rel. Error: 0.101%
	Mesh: 462466.116 		 Rel. Error: 0.168%


## 2. Surface Calculation

The second Minkowski functional (surface area) can be computed as:
$$ M_1(X) = \int_{\delta X} ds $$

Again, there are multiple ways to compute the surface area of the solid/pore boundary. Here, we compare:

1. Quantimpy (voxelized), and
2. Surface Mesh method (Skimage):

In [7]:
def compute_surface_area(image, smoothing=None):
    # Quantimpy
    quantimpy_surface_area = 8 * mk.functionals(image.astype(bool))[1]
    
    # Mesh Surface Area
    verts, faces, normals, values = skimage.measure.marching_cubes(image, level = smoothing)
    mesh_surface_area = skimage.measure.mesh_surface_area(verts, faces)
    
    return quantimpy_surface_area, mesh_surface_area

surface_area_measures = compute_surface_area(sphere)
_, surface_area_measures_blocky = compute_surface_area(sphere, smoothing=0)
surface_area_errors = compute_error(surface_area_analytical, surface_area_measures)
surface_area_errors_blocky = compute_error(surface_area_analytical, [surface_area_measures_blocky])

print("Surface Area Calculations:")
print(f"\tAnalytical Solution: {surface_area_analytical:0.3f}")
print(f"\tQuantimpy: {surface_area_measures[0]:0.3f} \t\t Rel. Error: {surface_area_errors[0]:0.3f}%")
print(f"\tMesh (Smoother): {surface_area_measures[1]:0.3f} \t Rel. Error: {surface_area_errors[1]:0.3f}%")
print(f"\tMesh (Blockier): {surface_area_measures_blocky:0.3f} \t Rel. Error: {surface_area_errors_blocky[0]:0.3f}%")

Surface Area Calculations:
	Analytical Solution: 28952.918
	Quantimpy: 28894.807 		 Rel. Error: 0.201%
	Mesh (Smoother): 31435.471 	 Rel. Error: 8.574%
	Mesh (Blockier): 31954.684 	 Rel. Error: 10.368%


## 3. Integral Mean Curvature Calculation

The third Minkowski functional (integral mean curvature area) can be computed as:
    $$ M_2(X) = \int_{\delta X} \frac{1}{2} \left[ \frac{1}{R_1} + \frac{1}{R_2} \right] ds $$
 

3D curvature measurements are not trivial to compute. Though there are many methods to compute the curvature, we will stick with the built in functions available in Quantimpy and the Trimesh library. 

1. Quantimpy (voxelized), and
2. Surface Mesh method (Implemented with Trimesh):

In [8]:
def compute_mean_curvature(image):
    # Quantimpy
    quantimpy_mean_curvature = 2*np.pi**2 * mk.functionals(image.astype(bool))[2]
    
    # Mesh Surface Area
    trimesh_sphere = trimesh.creation.icosphere(radius=48)
    mesh_mean_curvature = trimesh_sphere.integral_mean_curvature
    
    return quantimpy_mean_curvature, mesh_mean_curvature

mean_curvature_measures = compute_mean_curvature(sphere)
mean_curvature_errors = compute_error(mean_curvature_analytical, mean_curvature_measures)

print("Mean Calculations:")
print(f"\tAnalytical Solution: {mean_curvature_analytical:0.3f}")
print(f"\tQuantimpy: {mean_curvature_measures[0]:0.3f} \t Rel. Error: {mean_curvature_errors[0]:0.3f}%")
print(f"\tMesh: {mean_curvature_measures[1]:0.3f} \t\t Rel. Error: {mean_curvature_errors[1]:0.3f}%")


Mean Calculations:
	Analytical Solution: 603.186
	Quantimpy: 604.143 	 Rel. Error: 0.159%
	Mesh: 602.227 		 Rel. Error: 0.159%


## 4. Euler Characteristic

The fourth Minkowski functional (Gaussian curvature) can be computed as:
    $$ M_3(X) = \int_{\delta X} \left[ \frac{1}{R_1R_2} \right] ds $$
    
The Gauss-Bonnet theorem links the Gaussian Curvature to the Euler characteristic (or Euler number) by:

$$ 4\pi\chi = M_3(X)$$


Here, we compute the Euler number using:
1. Quantimpy, and
2. Skimage

In [15]:
def compute_euler_number(image):
    # Quantimpy
    quantimpy_euler_number = 4*np.pi/3 * mk.functionals(image.astype(bool))[3]
    
    # Mesh Surface Area
    skimage_euler_number = skimage.measure.euler_number(image, connectivity=3)
    
    return quantimpy_euler_number, skimage_euler_number

euler_number_measures = compute_euler_number(sphere)
euler_number_errors = compute_error(euler_analytical, euler_number_measures)

print("Euler Number Calculations:")
print(f"\tAnalytical Solution: {euler_analytical:0.3f}")
print(f"\tQuantimpy: {euler_number_measures[0]:0.3f} \t Rel. Error: {euler_number_errors[0]:0.3f}%")
print(f"\tSkimage: {euler_number_measures[1]:0.3f} \t\t Rel. Error: {euler_number_errors[1]:0.3f}%")


Euler Number Calculations:
	Analytical Solution: 1.000
	Quantimpy: 1.000 	 Rel. Error: 0.000%
	Skimage: 1.000 		 Rel. Error: 0.000%


# Minkowski Functionals for our samples

In the following cell, we compute the Minkowski functionals for the four samples we previously examined. For sake of simplicity, we will only use the Quantimpy library here.

In [19]:
def get_quantimpy_mf(image):
    mf0, mf1, mf2, mf3 = mk.functionals(image.astype(bool))
    mf1 *= 8
    mf2 *= 2*np.pi**2
    mf3 *= 4*np.pi/3
    return [mf0, mf1, mf2, mf3]

widget = widgets.Dropdown()
widget.options=['Beadpack', 'Sandpack', 'Mt. Gambier Limestone', 'Castlegate Sandstone']
widget.description='Sample: '
widget.value='Beadpack'


display(widget)



def compute_sample_mf(*args):
    
    sample_dict = {'Sandpack': segmented_lrc32,
                  'Mt. Gambier Limestone': segmented_Gambier, 
                   'Beadpack': segmented_bead_pack_512, 
                   'Castlegate Sandstone': segmented_castle_512}
    

    
    fig, ax = plt.subplots(nrows=1, ncols=1,figsize=(8,8))
    
    clear_output(wait=True)
    display(widget)
    
    sample = sample_dict[widget.value]
    
    ax.imshow(sample_dict[widget.value][0,:,:], cmap='gray')
    ax.set_title(widget.value,fontsize=14)
    mfs = get_quantimpy_mf(sample)
    
    print('Minkowski Functionals:')
    print(f'\tVolume: {mfs[0]:0.3f}')
    print(f'\tSurface Area: {mfs[1]:0.3f}')
    print(f'\tIntegral Mean Curvature: {mfs[2]:0.3f}')
    print(f'\tEuler Number: {mfs[3]:0.3f}')
    
compute_sample_mf()
widget.observe(compute_sample_mf, 'value')


    


Dropdown(description='Sample: ', options=('Beadpack', 'Sandpack', 'Mt. Gambier Limestone', 'Castlegate Sandsto…

Minkowski Functionals:
	Volume: 50531799.000
	Surface Area: 5308047.107
	Integral Mean Curvature: 23331749663905267712.000
	Euler Number: -457.375
